# Analysis of District Standardized Test Results

In [26]:
# Dependencies and Setup
import pandas as pd

# Files to load
students_filepath = "../Resources/students_complete.csv"
schools_filepath = "../Resources/schools_complete.csv"

# Read files and create pandas dataframes
students_df = pd.read_csv(students_filepath, encoding = "UTF-8")
schools_df = pd.read_csv(schools_filepath, encoding = "UTF-8")

# Merge data into one dataframe
student_results_df = pd.merge(students_df, schools_df, on="school_name", how="left" ) 

## District Summary
* Overview of key district metrics
  * Total Schools
  * Total Students
  * Total Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of Math and Reading)

In [27]:
# Calculate district summary statistics

# Find number of schools
unique_schools = len(student_results_df["school_name"].unique())

# Find total students
total_students = len(student_results_df["gender"].sum())

# Find total budget
total_budget = schools_df["budget"].sum()

# Find average math score
math_score_avg = student_results_df["math_score"].mean()

# Find average reading score
reading_score_avg = student_results_df["reading_score"].mean()

# Calculate percent of students passing math where passing >=70 
passing_math = (student_results_df["math_score"]>=70).value_counts(normalize=True)
math_pass_pct = passing_math[True]*100

# Calculate percent of students passing reading where passing >=70 
passing_reading = (student_results_df["reading_score"]>=70).value_counts(normalize=True)
reading_pass_pct = passing_reading[True]*100

# Calculate percent of students passing overall
overall_pass_pct = (math_pass_pct + reading_pass_pct)/2

In [28]:
# Create district summary dataframe
district_summary = [{"Total Schools": unique_schools,
                     "Total Students": total_students,
                     "Total Budget": total_budget,
                     "Average Math Score": math_score_avg,
                     "Average Reading Score": reading_score_avg,
                     "% Passing Math": math_pass_pct,
                     "% Passing Reading": reading_pass_pct,
                     "% Passing Overall": overall_pass_pct
                       }]
district_summary_df = pd.DataFrame(district_summary)

# Format dataframe
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,"39,170","$24,649,428",78.985371,81.87784,74.980853,85.805463,80.393158


# School Summary
* Overview of key metrics about each school
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of Math and Reading)

In [29]:
# Calculate school summary statistics

# Create groupby school name dataframe
school_summary_df = student_results_df.groupby("school_name")

# Calculate per student budget
budget_per_student = school_summary_df["budget"].sum()/school_summary_df["size"].sum()

# Find average math score
math_school_avg = school_summary_df["math_score"].mean()

# Find average reading score
reading_school_avg = school_summary_df["reading_score"].mean()

# Calculate percent of students passing math reading and overall in each school where passing >=70
school_passing_math = []
school_passing_reading = [] 
school_passing_overall = []

for school, group in school_summary_df:
    
    school_passing_math_count = (group["math_score"]>=70).value_counts(normalize=True)
    school_pass_math_pct = school_passing_math_count[True]*100
    school_passing_math.append(school_pass_math_pct)

    school_passing_reading_count = (group["reading_score"]>=70).value_counts(normalize=True)
    school_pass_reading_pct = school_passing_reading_count[True]*100   
    school_passing_reading.append(school_pass_reading_pct)

    school_pass_overall_pct = (school_pass_math_pct + school_pass_reading_pct)/2
    school_passing_overall.append(school_pass_overall_pct)  


In [30]:
# Create school summary dataframe using groupby school name dataframe
school_summary2 = {"Per Student Budget": budget_per_student,
                  "Average Math Score": math_school_avg,
                  "Average Reading Score": reading_school_avg,
                  "% Passing Math": school_passing_math,
                  "% Passing Reading": school_passing_reading,
                  "% Passing Overall": school_passing_overall
                 }
school_summary2_df = pd.DataFrame(school_summary2)

# Merge schools dataframe with summary dataframe
school_summary_final_df = pd.merge(schools_df, school_summary2_df, on="school_name", how="outer")

# Cleanup dataframe and format
school_summary_fin_df = school_summary_final_df 
school_summary_fin_df = school_summary_fin_df.drop(columns = ["School ID", ])
school_summary_fin_df = school_summary_fin_df.set_index("school_name")
school_summary_fin_df = school_summary_fin_df.rename(columns={"type": "School Type", 
                                                              "size": "Total Students", 
                                                              "budget": "Total School Budget"})
school_summary_fin_df["Total School Budget"] = school_summary_fin_df["Total School Budget"].map("${:,.2f}".format)
school_summary_fin_df["Per Student Budget"] = school_summary_fin_df["Per Student Budget"].map("${:.2f}".format)
school_summary_fin_df.index.name = None
school_summary_fin_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,95.854628,94.860875
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270


## Top Performing Schools (By Passing Rate)
* Display top five performing schools by passing rate

In [31]:
# Sort dataframe to display top five performing schools by passing rate
school_summary_top_df = school_summary_fin_df.sort_values("% Passing Overall", ascending=False)
school_summary_top_df.index.name = None
school_summary_top_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)
* Display bottom five performing schools by passing rate

In [32]:
# Sort dataframe to display bottom five performing schools by passing rate
school_summary_top_df = school_summary_fin_df.sort_values("% Passing Overall")
school_summary_top_df.index.name = None
school_summary_top_df.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade
* Display average math scores by grade in each school

In [33]:
# Create groupby dataframe by school and grade and find mean values
math_groupby = student_results_df.groupby(["school_name", "grade"])
math_mean_grade = math_groupby.mean()

# Create pivot table from dataframe and reindex by grade
math_averages_df = math_mean_grade.pivot_table("math_score", ["school_name"], "grade")
math_averages_df = math_averages_df[["9th", "10th", "11th", "12th"]]
math_averages_df

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Scores by Grade
* Display average reading scores by grade in each school

In [34]:
# Create groupby dataframe by school and grade and find mean values
reading_groupby = student_results_df.groupby(["school_name", "grade"])
reading_mean_grade = reading_groupby.mean()

# Create pivot table from dataframe and reindex by grade
reading_averages_df = reading_mean_grade.pivot_table("reading_score", ["school_name"], "grade")
reading_averages_df = reading_averages_df[["9th", "10th", "11th", "12th"]]
reading_averages_df

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by Student Spending
* Group data by student spending and display following metrics for each group
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of Math and Reading)

In [35]:
# Rename school summary dataframe to school size dataframe
spend_results_df = school_summary_final_df

# Create "bins" based on per student budget
spend_bins = [0, 585, 615, 645, 675]
spend_labels = ["<$585", "$585-615", "$615-645", "$645-675"]

# Bin data using bins created above
spend_results_df["Spending Ranges (Per Student)"] = pd.cut(spend_results_df["Per Student Budget"], 
                                        spend_bins, labels=spend_labels)

# Group by spending ranges and display
spend_results_df = spend_results_df.groupby("Spending Ranges (Per Student)")
spend_results_df = spend_results_df[["Average Math Score",
                                     "Average Reading Score",
                                     "% Passing Math",
                                     "% Passing Reading",
                                     "% Passing Overall"]]
spend_results_df.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size
* Group data by school size and display following metrics for each group
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of Math and Reading)

In [36]:
# Rename school summary dataframe to school size dataframe
school_size_results_df = school_summary_final_df

# Create "bins" based on per student budget
school_size_bins = [0, 1000, 2000, 5000]
school_size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Bin data using bins created above
school_size_results_df["School Size"] = pd.cut(school_size_results_df["size"], 
                                        school_size_bins, labels=school_size_labels)

# Group by spending ranges and display
school_size_results_df = school_size_results_df.groupby("School Size")
school_size_results_df = school_size_results_df[["Average Math Score",
                                                 "Average Reading Score",
                                                 "% Passing Math",
                                                 "% Passing Reading",
                                                 "% Passing Overall"]]
school_size_results_df.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type
* Group data by school type and display following metrics for each group
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of Math and Reading)

In [37]:
school_type_results_df = school_summary_final_df.groupby("type")
school_type_results_df = school_type_results_df[["Average Math Score",
                                                 "Average Reading Score",
                                                 "% Passing Math",
                                                 "% Passing Reading",
                                                 "% Passing Overall"]]

school_type_results_df.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


# Conclusion

   In this analysis, we looked at standardized test results in reading and math for high school students (grades 9 - 12) in a large school district. The population consisted of every single high school student in the district. Our data also included information about school size and school budget.   

   Based on the data, we can see that the percentage of students passing math in standardized tests is lower than the percentage passing reading for all schools in the district and significantly lower in bottom performing schools, where the percentage difference is around 15%. As such, it may be prudent to increase the resources used for math education or re-evaluate the math curriculum. More in depth analysis would be required to determine the best course of action.
   
   Charter school students in this district are also better performers on standardized tests than district school students, both in overall standardized test performance as a group and on a per school basis: the top five performing schools in the district are charter schools and the bottom five performing schools are district schools. This does not seem to be due to higher funding for charter schools, as the average per student budget for charter schools is lower than the average per student budget for district schools. Significant research into other differences between these two groups could facilitate understanding the reasons for this divergence, including pupils per teacher, school and class structure, etc.
   
   Finally, this analysis suggests the optimum school size for best performance on standardized tests to be between 1000 and 2000 students. This conclusion is severely limited by the information available for this analysis, however, conducting more research into school size and performance could lead to robust evidence to support the implementation of requirements for schools in this size range. 